# Merge all auctions into one

Combines several datasets and do some simple analyses.  


In [1]:
import sys
import os
import re
import json

In [2]:
with open('../assets/drz-settings-current.json', 'r') as fid:
    cfg = json.load(fid)
print(cfg['AUCTION'])

OPBOD = cfg['AUCTION']['kind'] == 'opbod'
AUCTION_ID = cfg['AUCTION']['id']
DATE = cfg['AUCTION']['date']
DATA_DIR = cfg['FILE_LOCATION']['data_dir']
#REGEX_DIR = cfg['FILE_LOCATION']['regex_dir']
auction_month = DATE[:4] + '-' + DATE[4:6]
if cfg['AUCTION']['kind'] == 'inschrijving':
    month_counter = re.sub('(-)(\d{2})', '\g<1>', AUCTION_ID)[5:8]
elif cfg['AUCTION']['kind'] == 'opbod':
    month_counter = re.sub('(-)(\d{2})(\d{2})', '-\g<2>', AUCTION_ID)[5:8]

#sys.path.insert(0, cfg['FILE_LOCATION']['code_dir'])

VERBOSE = int(cfg['GENERAL']['verbose'])
SAVE_METHOD = cfg['GENERAL']['save_method']


{'kind': 'inschrijving', 'id': '2024-0007', 'date': '20240412'}


In [3]:
if SAVE_METHOD == 'skip_when_exist':
    do_save = lambda fn: not(os.path.isfile(fn))
elif SAVE_METHOD == 'always_overwrite':
    do_save = lambda _: True
elif SAVE_METHOD == 'skip_save':
    do_save = lambda _: False
else:
    raise NotImplementedError(f'SAVE_METHOD: {SAVE_METHOD} not implemented')

In [4]:
TAG_SINGLE = "nbconvert_instruction:remove_single_output"

In [5]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', '2021-09-09', '2021-10-10', '2021-11-11', '2021-12-12', 
                    '2022-01-01', '2022-02-02', '2022-04-04', '2022-05-05', '2022-06-06', '2022-08-08', '2022-09-09', '2022-10-10', '2022-11-11', '2022-12-12',
                    '2023-01-01', '2023-02-02', '2023-03-03', '2023-04-04', '2023-05-05', '2023-06-06', '2023-07-07', '2023-08-08', '2023-09-09', '2023-10-10',  '2023-11-11',  '2023-12-12',
                    '2024-01-01', '2024-02-02', '2024-03-03', '2024-04-04'
                   ]
else:    
    AuctionDates = ['2014-10', '2014-11', '2014-12', # old!
                    '2015-01', '2015-02', '2015-03', # old!
                    '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                    '2021-06', '2021-06-16', '2021-07-07',  '2021-07-17', '2021-08-08', '2021-08-18', '2021-09-09', '2021-09-19', '2021-10-10', '2021-10-20', '2021-11-11', '2021-11-21', '2021-12-12', '2021-12-22',
                    '2022-01-01', '2022-01-21', '2022-02-02', '2022-02-22', '2022-03-03', '2022-03-23', '2022-04-04', '2022-04-24', '2022-05-05', '2022-05-25', '2022-06-06', 
                    '2022-07-07', '2022-07-27', '2022-08-08', '2022-08-28', '2022-09-09', '2022-09-29', '2022-10-10', '2022-10-30', '2022-11-11', '2022-11-31', '2022-12-12', '2022-12-32',
                    '2023-01-01', '2023-01-02', '2023-02-03', '2023-02-04', '2023-03-05', '2023-03-06', '2023-04-07', '2023-04-08', '2023-05-09', '2023-05-10', '2023-06-11', '2023-06-12',
                    '2023-07-13', '2023-07-14', '2023-08-15', '2023-08-16', '2023-09-17', '2023-10-19', '2023-10-20', '2023-11-21', '2023-11-22', '2023-12-23', '2023-12-24',
                    '2024-01-01', '2024-01-02', '2024-02-03', '2024-02-04', '2024-03-05', '2024-03-06', '2024-04-07'
                   ]  
if DATE[:6] not in [''.join(d.split('-')[:2]) for d in AuctionDates]:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load reference tables

In [7]:
name_table = pd.read_excel(f"{cfg['FILE_LOCATION']['code_dir']}/assets/fieldname-lookuptable.ods", sheet_name='preferred_name', header=0, skipfooter=1, dtype=str)
name_table = name_table.dropna(how='all', axis=0).dropna(how='all', axis=1)
rename_dict={}
for i,r in name_table.iterrows():
    r=r.dropna()
    k = r.iloc[0]
    for option in r.iloc[1:]:
        rename_dict[option] = k

print(len(rename_dict))

541


In [8]:
# some have trailing _X-s as counter
# if option does not have counter use _1
pat = '(_X)+$'
for from_name, to_name in rename_dict.items():
    # count nr of _X
    lvl = [None,None]
    M = re.search(pat, from_name); 
    lvl[0] = int((M.end() - M.start())/2) if M is not None else 0
    M = re.search(pat, to_name); 
    lvl[1] = int((M.end() - M.start())/2) if M is not None else 0

    # if disbalance assume _1 
    # rename last _X to _1 repeat for diff in _X
    if lvl[0]<lvl[1]:
        for i in range(lvl[1]-lvl[0]):
            to_name = re.sub('_X(_1)?$', '_1\g<1>', to_name)
        rename_dict[from_name] = to_name

In [9]:
rename_dict_with_counter = {}
for k,v in rename_dict.items():
    pat = '_X(_\d)?$'
    if re.search(pat, v) is not None:
        # should have equal nr of _X
        n = [None,None]
        M = re.search(pat, k)
        n[0] = len(re.findall('_X', M[0]))
        M = re.search(pat, v)
        n[1] = len(re.findall('_X', M[0]))
        assert n[0] == n[1]
        rename_dict_with_counter[k] = v

for k in rename_dict_with_counter.keys():
    del rename_dict[k]

print(len(rename_dict), len(rename_dict_with_counter))

462 79


### Load all data

Auctions are saved monthly in a .pkl file

In [10]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = f'{DATA_DIR}/auctions/enriched-results/rdw-data-{AuctionDate}-opbod.pkl'

    else:    
        fn = f'{DATA_DIR}/auctions/enriched-results/rdw-data-{AuctionDate}.pkl'

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    
    # flatten column levels
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.map(lambda x: '_'.join(x))# if x[0] != 'drz' else '_'.join(x[1:]))
        
    # store in dict
    data[AuctionDate] = df
    
    
# When there are more auctions in a single month
if not OPBOD:
    # HACK ALERT!!
    data['2021-06-16'].index = data['2021-06-16'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-06'].index = data['2021-06'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    
    # matrix of auction date
    auct_month = np.array([[int(k.split('-')[0]), int(k.split('-')[1])] for k in data.keys()])
    auct_month = np.unique(auct_month, axis=0)

    # 2021-07 there is a systematic index change
    for m in auct_month[((auct_month[:,0] == 2021) & (auct_month[:,1] >= 7)) | (auct_month[:,0] == 2022)]:
        for c in range(2):

            # multiplier
            #   2021: 10
            #   2022, 2023: 20
            if m[0] == 2021:
                multiplier = 10
            elif m[0] == 2022:
                multiplier = 20
            else:
                raise NotImplementedError

            # data key
            k = f'{m[0]}-{m[1]:02.0f}-{m[1]+(c*multiplier):02.0f}'
            if k not in data.keys():
                # skip if not exist
                continue

            # translation
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))

            if VERBOSE > 0:
                print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
    # as of 2023 auctions are just incremented by 1
    for m in auct_month[(auct_month[:,0] >= 2023)]:
        for k in [k for k in data.keys() if k.startswith(f'{m[0]}-{m[1]:02.0f}')]:
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))
            print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
            
            
        
if VERBOSE > 0:
    display({'text/plain': '\nlast rows of last file'}, raw=True, metadata={"tags":(TAG_SINGLE, )})
    display(data[AuctionDates[-1]].tail(), metadata={"tags":(TAG_SINGLE, )})

load data
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2014-10.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2014-11.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2014-12.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2015-01.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2015-02.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2015-03.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2017-03.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2017-04.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2017-05.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2017-06.pkl
/home/tom/bin/satdat

### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [11]:
# Change type of early auctions for images
if VERBOSE > 0:
    print('Fields with images look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images', 'drz__images'])
        ]
        example = data[k][im_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_)

if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        df_ = data[k].copy() # avoid performance warning
        df_['drz__Images'] = df_['Image'].apply(eval)
        data[k] = df_.drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        df_ = data[k].copy()
        df_['drz__Images'] = df_['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
        data[k] = df_.drop(columns=['Images'])
    

Fields with images look like this:



,column,type,first row
2014-10,Images,<class 'list'>,0
2014-11,Images,<class 'list'>,0
2014-12,Images,<class 'list'>,0
2015-01,Images,<class 'list'>,0
2015-02,Images,<class 'list'>,0
2015-03,Images,<class 'list'>,0
2017-03,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14883593...
2017-04,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14906980...
2017-05,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14931056...
2017-06,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14962307...


In [12]:
VERBOSE = 1
# Change dicttype of early auctions to regular fields
if VERBOSE > 0:
    print('rdwinfo look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first length'])
    for k in data:
        ri_cols = data[k].columns[
            data[k].columns.str.lower().isin(['rdwinfo'])
        ]
        if ri_cols.shape[0] == 0:
            continue
        example = data[k][ri_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = ri_cols[0]
        df_.loc[k, 'first length'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_.dropna(how='all'))

if OPBOD:
    NotImplementedError
else:
    

    # Convert list to dict first
    # df_.groupby('type').get_group("<class 'list'>").index
    for k in ['2017-04',
              '2017-05',
              '2018-07',
              '2018-09',
              '2018-10']:
        df_ = data[k].copy()
        assert all(df_['rdwinfo'].apply(lambda x: len(x) if isinstance(x, list) else 0)<=1), 'Which item in list should I pick'
        df_['rdwinfo'] = df_['rdwinfo'].apply(lambda x: x[0] if isinstance(x, list) else None)
        data[k] = df_

    # expand dict to DF columns
    # df_.groupby('type').get_group("<class 'dict'>").index
    for k in [
        '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
        '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
        '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
        '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
        '2019-03'
    ]:
        df_ = data[k].copy()
        columns = set(df_['rdwinfo'].apply(lambda x: list(x.keys()) if x is not None else []).sum())

        df_ri = pd.json_normalize(df_['rdwinfo'], sep='|')
        df_ri.index = df_.index
        df_ri.columns = df_ri.columns.str.split('|', expand=True)
        columns_with_suffix = df_ri.loc[:, (slice(None), '1')].columns.get_level_values(0)
        columns_wo_suffix = np.setdiff1d(df_ri.columns.get_level_values(0), columns_with_suffix)

        assert df_ri.columns.nlevels == 2, 'more splits in columns'
        new_col_names = dict()
        for c in df_ri.columns:
            if c[0] in columns_wo_suffix:
                new_col_names[c] = c[0]
            elif c[0] in columns_with_suffix:
                new_col_names[c] = c[0] + '_' + str(int(c[1])+1)
        df_ri.columns = new_col_names.values()
        df_ri = df_ri.add_prefix('rdw_')
        # kenteken_1 and _2 in set. reduce to one
        cols = [c for c in df_ri.columns if c.startswith('rdw_kenteken_')]
        if len(cols) > 0:
            reg_plate = df_ri.loc[:, cols].bfill(axis=1).ffill(axis=1)
            assert (reg_plate.fillna('').nunique(axis=1) > 1).any() == False, 'regs not equal'
            df_ri['rdw_kenteken'] = reg_plate.rdw_kenteken_1
            df_ri.drop(columns=cols, inplace=True)
        existing_cols = np.intersect1d(df_ri.columns, df_.columns).tolist()
        existing_cols += ['rdwinfo']
        data[k] = pd.merge(df_.drop(columns=existing_cols), df_ri, left_index=True, right_index=True)


rdwinfo look like this:



,column,type,first length
2017-03,rdwinfo,<class 'dict'>,55
2017-04,rdwinfo,<class 'list'>,1
2017-05,rdwinfo,<class 'list'>,1
2017-06,rdwinfo,<class 'dict'>,53
2017-07,rdwinfo,<class 'dict'>,38
2017-08,rdwinfo,<class 'dict'>,45
2017-09,rdwinfo,<class 'dict'>,54
2017-10,rdwinfo,<class 'dict'>,46
2017-11,rdwinfo,<class 'dict'>,52
2017-12,rdwinfo,<class 'dict'>,54


In [13]:
# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column names that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'drz__Draw',
            'nr':'drz__LotNr',
            'raw':'drz__Raw_text',
            'misc':'drz__SupInfo',
            'footnote':'drz__Note',
            'jfq':'drz__jfc',
        })
        

In [14]:
for auct_date, df_orig in data.items():

    df = df_orig.copy()
    # standardize column names
    df.rename(columns=rename_dict, inplace=True)
    # with counters
    rn_dict = {}
    for k,v in rename_dict_with_counter.items():
        search_pat = re.sub('(_X)+(_\d+)?$', '_', k)
        sel_rn = df.columns.str.startswith(search_pat)
        if any(sel_rn):
            for col in df.columns[sel_rn]:
                if ('type1' in v) and ('type_1' in k):
                    counter = re.search('(_type_1)?((_\d+)+)$', col)[2] # ignore if ..type_1.. is in name
                else:
                    counter = re.search('(_\d+)+$', col)[0]
                newv = re.sub('(_X)+(_\d+)*$', counter + '\g<2>', v)
                rn_dict[col] = newv
                
                
    df.rename(columns=rn_dict, inplace=True)

    # this introduces duplicates in column names
    # drop second, keep first
    vc = df.columns.value_counts()
    duplicate_cols = vc[vc>1].index
    for c in duplicate_cols:
        # get column number
        idx = np.where(df.columns.get_loc(c))[0]
        assert len(idx) == 2 # for now only do first/second
        # update first col
        df.iloc[:,idx[0]] = df.iloc[:,idx[0]].combine_first(df.iloc[:,idx[1]])
        # drop second col
        df = df.iloc[:, [j for j in range(len(df.columns)) if j != idx[1]]]

    data[auct_date] = df
    del df_orig

# Merge all auctions into one data frame

In [15]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

In [16]:
if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    # deprecated: col_filled = all_data.isna().all(level=0, axis=0) == False
    col_filled = all_data.notna().groupby(level=0).any()

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_source, df__ in df_.groupby(df_.index.str.split('_').str.get(0)):
        print(_by_source)
        for g,df___ in df__.groupby('first'):          
            if g == col_exist.index[0]:
                print(f'First data in {g}: {df___.shape[0]:.0f}')
                continue
            else:
                print(f'Fields added in {g}', end= ' ')
            with pd.option_context("display.max_rows", 9999):
                display(df___)#.sort_index())#.sort_values(by='last_filled', ascending=False))

drz
First data in 2014-10: 72
Fields added in 2017-11 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__used_parts,2017-11,2017-11,1,2017-11,1,0.925926,0.925926


Fields added in 2018-01 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__import22_btw21,2018-01,2018-07,2,2018-07,2,1.886792,1.886792
drz__import27_btw21,2018-01,2018-07,2,2018-07,2,1.886792,1.886792


Fields added in 2018-04 

...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_motor_uitvoering_brandstof_emissie_co2_gecombineerd_3_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_emissie_co2_gecombineerd_4_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_verbruik_gecombineerd_3_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_verbruik_gecombineerd_4_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_elektrisch_verbruik_gecomb_3_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_elektrisch_verbruik_gecomb_4_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_emissie_koolmonoxide_esc_test_3_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_emissie_koolmonoxide_esc_test_4_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_emissie_stikstofoxiden_esc_test_3_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0
rdw_motor_uitvoering_brandstof_emissie_stikstofoxiden_esc_test_4_1,2024-02-04,2024-02-04,1,2024-04-07,0,25.0,0.0


Fields added in 2024-03-05 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_1_13,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_1_14,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_2_13,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_2_14,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_3_14,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_4_12,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_1_13,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_1_14,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_2_13,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_2_14,2024-03-05,2024-03-05,1,2024-03-05,1,33.333333,33.333333


Fields added in 2024-04-07 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_bijzonderheden_bijzonderheid_volgnummer_3,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_code_3,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_code_1_3,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_variabele_tekst_3,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_eenheid_3,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_versnellingsbak_uitvoering_volgnummer_2,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0
rdw_versnellingsbak_uitvoering_composite_key_2,2024-04-07,2024-04-07,1,2024-04-07,1,100.0,100.0


- - - -
# Continue with cars only

In [17]:
# select data
car=df[df.drz__LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    df_['sfx'] = df_.lot.apply(lambda x: 0 if x<9999 else int(str(x)[-2:]))
    with pd.option_context('display.max_rows', 999):
        display(df_.groupby(['year', 'month']).agg({'lot': ['min', 'max'], 'sfx': ['min', 'max', 'count']}).sort_index())
    

display(car.tail(), metadata={"tags":(TAG_SINGLE, )})

lot         sfx          
               min     max min max count
year month                              
2014 10       2000    7312   0   0   136
     11       2400    8198   0   0   133
     12       2000    7306   0   0   147
2015 1        2402    8318   0   0   134
     2        2000    7318   0   0   138
     3        2400    8312   0   0   157
2017 3        2000    8318   0   0   149
     4        2000    7309   0   0   133
     5        2200    8025   0   0    75
     6        2000    7311   0   0   140
     7        2200    8178   0   0   106
     8        2001    7304   0   0   143
     9        2200    8187   0   0   133
     10       2000    7171   0   0    95
     11       2200    8305   0   0   158
     12       2000    7308   0   0   136
2018 1        2200    8170   0   0   134
     2        2000    7184   0   0   138
     3        2200    8338   0   0   186
     4        2000    7336   0   0   126
     5        2200    8226   0   0   186
     6        2000    7315   0   0   166
     7        2200    8311   0   0   132
     8        2000    7328   0   0   152
     9        2200    8183   0   0    89
     10       2000    7319   0   0    87
     11       2200    8326   0   0   127
     12       2000    7361   0   0   133
2019 1        2201    8329   0   0   135
     2        2001    7264   0   0   176
     3        2200    8318   0   0   145
     4        2000    7314   0   0   170
     5        2200    8352   0   0   158
     6        2201    9158   0   0    98
     7        2200    7254   0   0   144
     8        2200    8172   0   0    96
     9        2200    9269   0   0   156
     10       2400    7200   0   0   109
     11       2200    8213   0   0   125
     12       2200    9061   0   0    75
2020 1        2200    7205   0   0   126
     2        2200    8238   0   0   129
     3        2200    9152   0   0    65
     6        2200    7269   0   0   160
     7        2200    9271   0   0   145
     8        2200    7286   0   0   187
     9        2200    8236   0   0   111
     10       2200    7266   0   0   129
     11       2200    8260   0   0   121
     12       2201    7306   0   0   168
2021 1        2200    8123   0   0   104
     2        2200    7156   0   0   129
     3        2200    8189   0   0   148
     4        2200    7220   0   0   173
     5        2200    8158   0   0   126
     6      220006  808316   6  16   191
     7      201217  900007   7  17   152
     8      260018  812918   8  18   143
     9      220009  808919   9  19   135
     10     260110  814920  10  20   181
     11     262411  805621  11  21    97
     12     260012  807722  12  22   171
2022 1      220021  805621   1  21   115
     2      240002  810122   2  22   193
     3      220003  807723   3  23   132
     4      240004  805724   4  24    88
     5      260025  805125   5  25   103
     6      700006  712506   6   6    79
     7      260007  982727   7  27   112
     8      260008  803228   8  28    81
     9      265029  803429   9  29   106
     10     700030  807010  10  30    67
     11     260011  803311  11  31   116
     12     260032  706912  12  32    65
2023 1      260001  709701   1   2   115
     2      200004  712003   3   4   171
     3      260306  707706   5   6   111
     4      260007  810707   7   8    99
     5      261909  710209   9  10   123
     6      260111  710211  11  12   116
     7      300314  708513  13  14    93
     8      260015  708915  15  16   132
     9      260017  705817  17  17    43
     10     260019  718219  19  20   189
     11     260121  712421  21  22   172
     12     200023  708723  23  24   118
2024 1      240002  710201   1   2   116
     2      220004  712404   3   4   122
     3      204106  710706   5   6   152
     4      240107  715607   7   7    89

# Save file

In [18]:
file_name = rf'{DATA_DIR}/cars-from-all-auctions.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if True or do_save(file_name): # always overwrite merged data file
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


/home/tom/bin/satdatsci/Saturday-Datascience/data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [19]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [20]:
file_name = f"{cfg['FILE_LOCATION']['code_dir']}/assets/example-table-of-cars.md"

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if do_save(file_name):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example, metadata={"tags":(TAG_SINGLE, )})
    


Skip. /home/tom/bin/satdatsci/Saturday-Datascience/code/assets/example-table-of-cars.md exists or saving is disabled in settings.


In [21]:
# idx = df.index.str.split('-', expand=True).droplevel(-1)
# is_last_auction = idx == idx[-1]
# subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
# VERBOSE

In [22]:
# show recurring lots (based on registration no.)

vc = df.drz__Reg.value_counts()

# drop obvious
vc.drop(['onbekend', ''], inplace=True, errors='ignore')

# reduce output
if VERBOSE < 2:
    # only of last auction
    idx = df.index.str.split('-', expand=True).droplevel(-1)
    is_last_auction = idx == idx[-1]
    subsel_reg = df.loc[is_last_auction, 'drz__Reg'].drop_duplicates().values
    vc = vc[vc.index.isin(subsel_reg)]
    
# show recurring
for r in vc[vc > 1].index:
    # title
    print(r, vc[r], end=': ')
    sel_reg = df.drz__Reg==r
    print(' - '.join(list(df.loc[sel_reg, [
        'drz__ItemBrand', 'drz__ItemType', 'rdw_gekentekende_voertuigen_datum_eerste_tenaamstelling_in_nederland'
    ]].drop_duplicates(ignore_index=True).iloc[0].astype(str))), end=' ')
    
    df_ = df.loc[sel_reg, list(df.columns.drop([
        'drz__Raw_text', 'rdwinfo', 'rdw_merk_registratie_datum', 'rdw_merk_registratie_datum_dt', 'rdw_merkcode', 'drz__Images', 'drz__Source'
    ], errors='ignore'))].sort_values(by='drz__Reg').T

    # convert index to nr of days since first
    ix = pd.Series(df_.columns.map(lambda x: pd.to_datetime('/'.join(x.split('-')[:2])))).diff().dt.days.copy()
    ix[0] = 0
    ix=ix.astype(int).astype(str)
    ix[0] = pd.to_datetime('/'.join(df_.columns[0].split('-')[:2])).strftime('%Y/%m')

    df_.columns=ix

    if (VERBOSE < 2) | (df_.shape[1] > 4):

        vc2 = df_.applymap(lambda x: str(x) if isinstance(x, list) else x).nunique(axis=1)
        print('')
        with pd.option_context('display.max_rows', 999):
            cols = list(vc2[vc2>1].index)
            cols += ['drz__OdoKM']
            cols += ['drz__OdoMLS']
            display(df_.loc[cols,:])
    elif df_.shape[1] > 2:
        display(df_.loc[['drz__Price'],:].T)
    else:
        print(*list(df_.columns))
        

VX-626-X 3: DODGE - ram 1500 - 20160721.0 


,2023/10,31,152
drz__Title,Kavel K2300197141,Kavel K2300217065,Kavel K2400077022
drz__lot_counter,7141.0,7065.0,7022.0
drz__Note,False,Dit kavel is vervallen op 3-11-2023 en wordt ...,False
drz__LotNr,K2300197141,K2300217065 kavel is vervallen,K2400077022
drz__LotType,Personenauto,Bedrijfswagen,Bedrijfswagen
drz__OdoKM,378.523,,378.523
drz__Mfdate,03.05.2016,,03.05.2016
drz__APKdate,03.05.2024,,03.05.2024
drz__Vin,1C6RR7MT5GS198333,,1C6RR7MT5GS198333
drz__disclaim5,True,False,True


R-290-NL 3: VOLKSWAGEN - t-roc - 20221013.0 


,2023/10,123,60
drz__Title,Kavel K2300207000,Kavel K2400047070,Kavel K2400077132
drz__Price,18000.0,18501.0,18746.0
drz__lot_counter,7000.0,7070.0,7132.0
drz__LotNr,K2300207000,K2400047070,K2400077132
rdw_gekentekende_voertuigen_TimeStamp,2023-10-21 08:04:53+00:00,2024-02-28 07:48:01+00:00,2024-04-12 18:05:08+00:00
rdw_ovi_TimeStamp,2023-10-21 08:09:37+00:00,2024-02-28 07:53:52+00:00,2024-04-12 18:10:56+00:00
rdw_assen_TimeStamp,2023-10-21 08:04:54+00:00,2024-02-28 07:48:01+00:00,2024-04-12 18:05:09+00:00
rdw_brandstof_TimeStamp,2023-10-21 08:04:55+00:00,2024-02-28 07:48:02+00:00,2024-04-12 18:05:10+00:00
rdw_carrosserie_TimeStamp,2023-10-21 08:04:55+00:00,2024-02-28 07:48:03+00:00,2024-04-12 18:05:10+00:00
rdw_keuringen_TimeStamp,2023-10-21 08:05:00+00:00,2024-02-28 07:48:08+00:00,2024-04-12 18:05:15+00:00


81-HDG-5 3: RENAULT - megane scenic - 20081030.0 


...

,2024/02,60
drz__Title,Kavel K2400047084,Kavel K2400077138
drz__Price,58000.0,52111.0
drz__lot_counter,7084.0,7138.0
drz__LotNr,K2400047084,K2400077138
drz__APKdate,28.02.2024,
rdw_gekentekende_voertuigen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:08+00:00
rdw_ovi_TimeStamp,2024-02-28 07:53:52+00:00,2024-04-12 18:10:56+00:00
rdw_assen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:09+00:00
rdw_brandstof_TimeStamp,2024-02-28 07:48:02+00:00,2024-04-12 18:05:10+00:00
rdw_carrosserie_TimeStamp,2024-02-28 07:48:03+00:00,2024-04-12 18:05:10+00:00


N-628-LL 2: AUDI - rs q3 sportback - 20211110.0 


,2024/02,60
drz__Title,Kavel K2400047043,Kavel K2400077140
drz__Price,46543.0,50000.0
drz__lot_counter,7043.0,7140.0
drz__LotNr,K2400047043,K2400077140
rdw_gekentekende_voertuigen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:08+00:00
rdw_ovi_TimeStamp,2024-02-28 07:53:52+00:00,2024-04-12 18:10:56+00:00
rdw_assen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:09+00:00
rdw_brandstof_TimeStamp,2024-02-28 07:48:02+00:00,2024-04-12 18:05:10+00:00
rdw_carrosserie_TimeStamp,2024-02-28 07:48:03+00:00,2024-04-12 18:05:10+00:00
rdw_keuringen_TimeStamp,2024-02-28 07:48:08+00:00,2024-04-12 18:05:15+00:00


5-TSP-78 2: SEAT - leon st - 20140508.0 


,2024/02,60
drz__Title,Kavel K2400047111,Kavel K2400077131
drz__lot_counter,7111.0,7131.0
drz__LotNr,K2400047111,K2400077131
rdw_gekentekende_voertuigen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:08+00:00
rdw_versnellingsbak_uitvoering_type_versnellingsbak_1,H,M
rdw_ovi_TimeStamp,2024-02-28 07:53:52+00:00,2024-04-12 18:10:56+00:00
rdw_assen_TimeStamp,2024-02-28 07:48:01+00:00,2024-04-12 18:05:09+00:00
rdw_brandstof_TimeStamp,2024-02-28 07:48:02+00:00,2024-04-12 18:05:10+00:00
rdw_carrosserie_TimeStamp,2024-02-28 07:48:03+00:00,2024-04-12 18:05:10+00:00
rdw_keuringen_TimeStamp,2024-02-28 07:48:08+00:00,2024-04-12 18:05:15+00:00
